In [2]:
from collocation_extraction import collect_verb_obl_obj, collect_core, collect_nmod, collect_amod, collect_comp, collect_appos, show_dependency
from text_preprocessing import preprocess_df
import pandas as pd
import spacy

In [3]:
df_labels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/final_labels.csv', index_col=0)

In [4]:
df_labels

,Emotion,Dehumanization,Mention,External ID,Created By,text,rating
1219,"так, присутня негативна",так,так,row_0.txt,snizannabotvin@gmail.com,"Всвязи с этим немного поправлю коллег ⤵️ ""Они...",4
1218,"ні, оцінка не присутня",ні,ні,row_1.txt,snizannabotvin@gmail.com,Литературный критик Галина Юзефович о новом ро...,4
1591,"так, присутня негативна",так,так,row_10.txt,snizannabotvin@gmail.com,Почему на базах неонацистов стоят языческие ис...,4
1198,"так, присутня негативна",так,так,row_100.txt,snizannabotvin@gmail.com,Группа добровольцев-медиков из Чеченской Респу...,4
3247,"ні, оцінка не присутня",ні,так,row_1000.txt,tutovadesign@gmail.com,"ВСУшники, переходите на сторону добра, у нас т...",5
...,...,...,...,...,...,...,...
3613,"ні, оцінка не присутня",ні,так,row_995.txt,tutovadesign@gmail.com,Утренний брифинг Минобороны России: ▪️ россий...,5
3612,"так, присутня негативна",ні,так,row_996.txt,tutovadesign@gmail.com,И понеслась мазепинщино-петлюровщино-бандеровщ...,5
4121,"так, присутня негативна",ні,так,row_997.txt,yevhen.marchenko91@gmail.com,Наш соратник по русскому движению Алексей Сели...,3
4120,"ні, оцінка не присутня",так,так,row_998.txt,yevhen.marchenko91@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,3


In [5]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, GPT2LMHeadModel, GPT2Tokenizer
import torch
import math

In [6]:
def translate(text, model, tokenizer, target_lang):
    encoded_text = tokenizer.encode(text, return_tensors="pt")
    translated_text = model.generate(encoded_text)
    return tokenizer.decode(translated_text[0], skip_special_tokens=True)

In [65]:
def calculate_perplexity(translated_sentence, model, tokenizer):
    if len(translated_sentence.strip()) < 3:  # Check if the sentence is empty or very short
        return -1  # Return a default perplexity value

    try:
        input_ids = tokenizer.encode(translated_sentence, return_tensors="pt")
        with torch.no_grad():
            outputs = model(input_ids[:, :-1])
            log_probs = torch.log_softmax(outputs.logits, dim=-1)
            token_probs = torch.gather(log_probs, 2, input_ids[:, 1:].unsqueeze(-1)).squeeze()
        perplexity = math.exp(-torch.mean(token_probs))
        return perplexity
    except Exception as e:
        print(f"Error with input '{translated_sentence}': {str(e)}")
        return -1  # Return a default perplexity value for problematic inputs


In [11]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, GPT2LMHeadModel, GPT2Tokenizer
import torch
import math

translation_model_name = "Helsinki-NLP/opus-mt-en-ru"

In [12]:
translation_model = MarianMTModel.from_pretrained(translation_model_name)

In [13]:
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)

In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
gpt_model_name = "gpt2"
gpt_tokenizer = GPT2Tokenizer.from_pretrained(gpt_model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(gpt_model_name).eval()

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [43]:
translation_model_name = "Helsinki-NLP/opus-mt-ru-en"
translation_model = MarianMTModel.from_pretrained(translation_model_name)
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/307M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/293 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

In [53]:
def translate(text, model, tokenizer, src_lang, tgt_lang):
    if not text.strip():  # Check if the input text is empty or contains only whitespace
        print("Empty input detected. Skipping translation.")
        return ""

    try:
        # Set the source and target languages
        tokenizer.src_lang = src_lang
        tokenizer.tgt_lang = tgt_lang

        encoded_text = tokenizer.encode(text, return_tensors="pt")
        translated_text = model.generate(encoded_text)
        return tokenizer.decode(translated_text[0], skip_special_tokens=True)
    except IndexError as e:
        print(f"Error encountered for sentence: '{text}'")
        print("Error details:", e)
        return ""


In [57]:
translation_model_name_ru = "Helsinki-NLP/opus-mt-en-ru"
translation_model_ru = MarianMTModel.from_pretrained(translation_model_name_ru)
translation_tokenizer_ru = MarianTokenizer.from_pretrained(translation_model_name_ru)

In [54]:
df_labels['text'].iloc[0]

'Всвязи с этим немного поправлю коллег ⤵️  "Они не будут начинать ядерный армагеддон" не только из-за вероятно созданной ими же диверсии в Польше, но и полагаю, не будут ничего делать, даже если мы нанесем осознанный удар ТЯО по Львовской рублевке или польскому логистическому центру, через который отправляют вооружение со всей европы в укро-рейх.'

In [55]:
result = translate(df_labels['text'].iloc[0], translation_model, translation_tokenizer, "ru", "en")

/Users/katerynaburovova/PycharmProjects/dehumanization/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [56]:
result

'In connection with this, I\'m going to correct a little bit of my colleagues, "They\'re not going to start a nuclear armagedon" not only because they probably created the same sabotage in Poland, but also think they won\'t do anything, even if we deliberately strike the TJO on the Lviv ruble or the Polish logistics center, through which they send weapons from all over the Europa in the cathedral.'

In [58]:
back = translate(result, translation_model_ru, translation_tokenizer_ru, "en", "ru")

/Users/katerynaburovova/PycharmProjects/dehumanization/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [59]:
back

'В связи с этим я собираюсь исправить кое-что из моих коллег: "Они не собираются начинать ядерное вооружение" не только потому, что они, вероятно, создали тот же саботаж в Польше, но и потому, что они думают, что ничего не сделают, даже если мы намеренно ударим TJO по Львовскому рублю или польскому логистическому центру, через который они посылают оружие со всей Европы в соборе.'

In [61]:
%%time
df_labels["translated_to_english"] = df_labels["text"].apply(
    lambda sentence: translate(sentence, translation_model, translation_tokenizer, "ru", "en")
)

/Users/katerynaburovova/PycharmProjects/dehumanization/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors


Error encountered for sentence: 'Ну почти., Антинавальный, БЛОК, Армен Гаспарян, Фаренгейт, Владимир Соловьёв, Илья Меркури, Якеменко, Политджойстик, Мария Бутина, Дмитрий Стешин, Александр Коц, Алексей Пушков, Сергей Карнаухов  , Земля наша, Юмор и Политика, Доктор Сосновский, Русский Демиург,  Вадим Манукян, Профессоръ Евстафьев, Железная логика Сергея Михеева,  Алексей Журавко, Андрей Манчук , Семён Пегов, Политспутник, Анастасия Попова, РУССТРАТ, Твиттерские беженцы , Архивариус, Кремлёвская прачка, Сергей Колясников, Орда, Несоциальная сеть, Маргарита Симоньян, Радио Лекух, Дробницкий, Евгений Примаков, Аббас Джума, Марина Ахмедова, Жёлтые СЛИВЫ, Елена Панина,   Мирослава Бердник,  Союз Блогеров Украины, Спецкор Курлаева, Владимир Корнилов,  Империя Очень Зла,  Мюсли Лаврова,  Выпускайте Кракена, Fuck you That's Why, Владимир Карасёв, Нюра Н. Берг, Breeg Time, Коза кричала , Мадам Секретарь , Игорь Молотов, Запрещённая Украина, Inside Donetsk, ВЕСЕЛОВСКИЙ, NEW KOBZAR, TheRussianNe

In [63]:
%%time
df_labels["translated_back_to_russian"] = df_labels["translated_to_english"].apply(
    lambda sentence: translate(sentence, translation_model_ru, translation_tokenizer_ru, "en", "ru")
)

/Users/katerynaburovova/PycharmProjects/dehumanization/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Empty input detected. Skipping translation.
Empty input detected. Skipping translation.
Empty input detected. Skipping translation.
CPU times: user 55min 8s, sys: 15min 45s, total: 1h 10min 54s
Wall time: 2h 49min 58s


In [64]:
df_labels

,Emotion,Dehumanization,Mention,External ID,Created By,text,rating,translated_to_english,translated_back_to_russian,perplexity
1219,"так, присутня негативна",так,так,row_0.txt,snizannabotvin@gmail.com,"Всвязи с этим немного поправлю коллег ⤵️ ""Они...",4,"In connection with this, I'm going to correct ...",В связи с этим я собираюсь исправить кое-что и...,5.489593
1218,"ні, оцінка не присутня",ні,ні,row_1.txt,snizannabotvin@gmail.com,Литературный критик Галина Юзефович о новом ро...,4,Literary critic Galina Yuzefović of Alexander ...,Литературный критик Галина Юзефович из нового ...,7.103959
1591,"так, присутня негативна",так,так,row_10.txt,snizannabotvin@gmail.com,Почему на базах неонацистов стоят языческие ис...,4,Why do neo-Nazis have pagan images on their ba...,Почему у неонацистов на их базе есть языческие...,3.788257
1198,"так, присутня негативна",так,так,row_100.txt,snizannabotvin@gmail.com,Группа добровольцев-медиков из Чеченской Респу...,4,A group of medical volunteers from the Republi...,Группа врачей-добровольцев из Чеченской Респуб...,19.975097
3247,"ні, оцінка не присутня",ні,так,row_1000.txt,tutovadesign@gmail.com,"ВСУшники, переходите на сторону добра, у нас т...",5,We have warm and tasty cookies.,У нас есть теплое и вкусное печенье.,113.599584
...,...,...,...,...,...,...,...,...,...,...
3613,"ні, оцінка не присутня",ні,так,row_995.txt,tutovadesign@gmail.com,Утренний брифинг Минобороны России: ▪️ россий...,5,The morning briefing by the Ministry of Defenc...,Утренний брифинг Министерства обороны Российск...,0.000000
3612,"так, присутня негативна",ні,так,row_996.txt,tutovadesign@gmail.com,И понеслась мазепинщино-петлюровщино-бандеровщ...,5,And there's a mazepinskin-pupil-bandering.,И там есть лазейка-пуф-бандинг.,0.000000
4121,"так, присутня негативна",ні,так,row_997.txt,yevhen.marchenko91@gmail.com,Наш соратник по русскому движению Алексей Сели...,3,"Our associate in the Russian movement, Alexei ...",Наш ассистент в российском движении Алексей Се...,0.000000
4120,"ні, оцінка не присутня",так,так,row_998.txt,yevhen.marchenko91@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,3,It's a good video from the 4th crew of the I.R...,Это хорошее видео от 4-й команды I.R.R. https:...,0.000000


In [66]:
%%time

df_labels["perplexity"] = df_labels["translated_back_to_russian"].apply(
    lambda sentence: calculate_perplexity(sentence, gpt_model, gpt_tokenizer)
)

Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 1024). Running this sequence through the model will result in indexing errors


Error with input 'В обозримом будущем пронацистский режим в Киеве мог бы получить ядерное оружие, а Россия была бы мишенью. Киевским властям, по различным каналам, было предложено не вступать в боевые действия, выводить войска из Донбасс, он не хотел этого делать. Наши ребята очень усердно работали на территории Украины. Киев саботировал мирное урегулирование кризиса. В обозримом будущем пронацистский режим в Киеве мог бы получить ядерное оружие, и его целью была бы Россия. Киевским властям по различным каналам было предложено не участвовать в боевых действиях, вывести войска из Донбасса в Российскую Федерацию, насколько они этого не хотят. Специальная операция шла хорошо, строго в соответствии с планом. Наши ребята на территории Украины работали упорно, Киев саботировали мирное урегулирование кризиса. В обозримом будущем пронацистский режим в Киеве мог бы получить ядерное оружие, цель была бы достигнута в России. Киевским властям по различным каналам было предложено не участвовать в б

In [68]:
columns_to_check = ['perplexity', 'translated_back_to_russian', 'translated_to_english']

mask = df_labels[columns_to_check].applymap(lambda x: x == "" or pd.isna(x)).any(axis=1)
df_clean = df_labels[~mask].copy()

In [71]:
df_clean = df_clean[df_clean['Dehumanization']!='не можу визначитись з правильною відповіддю'].copy()

In [73]:
df_clean['labels']=df_clean['Dehumanization'].apply(lambda x: 0 if x=='ні' else 1)

In [74]:
import numpy as np

pearson_correlation = df_clean["perplexity"].corr(df_clean["labels"], method="pearson")
print("Pearson correlation coefficient:", pearson_correlation)

Pearson correlation coefficient: -0.021382167692577024


In [75]:
spearman_correlation = df_clean["perplexity"].corr(df_clean["labels"], method="spearman")
print("Spearman correlation coefficient:", spearman_correlation)

Spearman correlation coefficient: -0.07398122831144073
